***Preprocessing the IRS data , aggregate by FIPS***

In [0]:
import pandas as pd
from google.colab import files 
import numpy as np

In [0]:
uploaded_irs = files.upload()


In [0]:
for fn1 in uploaded_irs.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn1, length=len(uploaded_irs[fn1])))

In [0]:
import io 
irs_df = pd.read_csv(io.StringIO(uploaded_irs['16zpallnoagi.csv'].decode('cp1252')))
irs_df.head(10) 

In [0]:
dt=irs_df
dt=dt.drop(['STATEFIPS', 'STATE','AGI_STUB'], axis=1)
dt.head()

![alt text](https://)**Remove zipcodes that are entered as 0**

In [0]:
dt.shape
dt = dt[dt.ZIPCODE != 0]
dt.shape
dt=dt[dt.ZIPCODE!=99999]
dt.head(10)

**Aggregated zipdata**

In [1]:
zip_data=dt.groupby(['ZIPCODE'],sort=False).mean()
zip_data.head(10)

NameError: ignored

In [0]:
zip_data.shape

**Load the HUD data for converting zip code to FIPS**

In [0]:
uploaded_hud = files.upload()


In [0]:
for fn2 in uploaded_hud.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn2, length=len(uploaded_hud[fn2])))

In [0]:
hud_df = pd.read_csv(io.StringIO(uploaded_hud['COUNTY_ZIP_122016.csv'].decode('cp1252')))
hud_df.head(10) 

In [0]:
zip_data.insert(0, "FIPS", "")
zip_data.reset_index(level=0, inplace=True)
zip_data.head(10)


In [0]:
zip_code_unmatched=[]
for iter in range(zip_data.shape[0]):
  search_value=zip_data.iloc[iter,0]
  sub_data_frame=hud_df.loc[hud_df['ZIP'] == search_value]
  if not sub_data_frame.empty:
    zip_data.iloc[iter,1]=sub_data_frame.iloc[0,0]
  else:
    zip_code_unmatched.append(search_value)
    
    

In [0]:
zip_data.head(20)

In [0]:
zip_code_unmatched

In [0]:
fips_code_unmatched=[4021,4017,4005,5103,8107,12107,12001,13279,21125,22075,25013,28011,28055,32013,35031,35039,40145,42071,48043,51107]

In [0]:
ind=0
for iter in zip_code_unmatched:
  bb=zip_data.loc[zip_data['ZIPCODE']==iter]
  ap=bb.index.values
  zip_data.iloc[ap,1]=fips_code_unmatched[ind]
  ind=ind+1
  


In [0]:
zip_data.head(10)

**Get rid of zip codes only use FIPS now**

In [0]:
zip_data=zip_data.drop(['ZIPCODE'], axis=1)

In [0]:
zip_data.head(10)

In [0]:
zip_data=zip_data.groupby(['FIPS'],sort=False).mean()
zip_data.head(10)
zip_data.shape

In [0]:
zip_data.to_csv('FIPSFilteredData.csv')
files.download('FIPSFilteredData.csv')